In [1]:
import os
import sys
import json
from pathlib import Path
from copy import deepcopy
import time
import inspect

from timeout_decorator import timeout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from pathlib import Path

from tqdm.notebook import tqdm

In [2]:
sys.path.insert(0, '../DSL')
import solvers as task_solvers
import verifiers as task_verifiers
from solver_class import Solver

sys.path.insert(0, '../visualization')
from visualization_utils import *

from fitness_scoring import *

In [3]:
data_path = Path('../data')
train_path = data_path / 'arc-agi_training_challenges.json'
train_sols_path = data_path / 'arc-agi_training_solutions.json'
eval_path = data_path / 'arc-agi_evaluation_challenges.json'
eval_sols_path = data_path / 'arc-agi_evaluation_solutions.json'
re_arc_path = data_path / 're-arc_tasks'

In [4]:
with open(train_path, 'r') as f:
    train_tasks = json.load(f)
with open(train_sols_path, 'r') as f:
    train_sols = json.load(f)
with open(eval_path, 'r') as f:
    eval_tasks = json.load(f)
with open(eval_sols_path, 'r') as f:
    eval_sols = json.load(f)
train_task_labels = sorted(train_tasks.keys())
eval_task_labels = sorted(eval_tasks.keys())

In [5]:
ast = lambda g: tuple(tuple(r) for r in g) # Converts grid to tuple format for DSL
asg = lambda t: list(list(r) for r in t) # Convert tuple back to grid

# Convert all train and eval examples to tuples for DSL
for train_label in train_task_labels:
    num_train = len(train_tasks[train_label]['train']) 
    num_test = len(train_tasks[train_label]['test'])
    for i in range(num_train):
        train_tasks[train_label]['train'][i]['input'] = ast(train_tasks[train_label]['train'][i]['input'])
        train_tasks[train_label]['train'][i]['output'] = ast(train_tasks[train_label]['train'][i]['output'])
    for i in range(num_test):
        train_tasks[train_label]['test'][i]['input'] = ast(train_tasks[train_label]['test'][i]['input'])
        train_sols[train_label][i] = ast(train_sols[train_label][i])
for eval_label in eval_task_labels:
    num_train = len(eval_tasks[eval_label]['train']) 
    num_test = len(eval_tasks[eval_label]['test'])
    for i in range(num_train):
        eval_tasks[eval_label]['train'][i]['input'] = ast(eval_tasks[eval_label]['train'][i]['input'])
        eval_tasks[eval_label]['train'][i]['output'] = ast(eval_tasks[eval_label]['train'][i]['output'])
    for i in range(num_test):
        eval_tasks[eval_label]['test'][i]['input'] = ast(eval_tasks[eval_label]['test'][i]['input'])
        eval_sols[eval_label][i] = ast(eval_sols[eval_label][i])

In [6]:
solver_functions = [Solver(inspect.getsource(getattr(task_solvers, f"solve_{label}"))) for label in train_task_labels]
verifier_functions = [Solver(inspect.getsource(getattr(task_verifiers, f"verify_{label}"))) for label in train_task_labels]

In [7]:
#del solver_functions['a64e4611'] # Extremely slow solver
#train_task_labels.remove('a64e4611')

In [8]:
train_scoring_results = {}
#test_scoring_results = {} # TODO: Compute fitness using test in/out as well?

for label in tqdm(eval_task_labels):
    task = eval_tasks[label]
    num_train = len(task['train'])
    num_test = len(task['test'])
    train_results = {}
    test_results = {}

    train_pairs = [{'input': task['train'][i]['input'],
                    'output': task['train'][i]['output']}
                   for i in range(num_train)]

    scoring_results = score_solvers_vs_tasks(solver_functions + verifier_functions, train_pairs, scoring_functions, solver_timeout = 0.5)
    scoring_results = {solver_name: score if score != None else 1.0 for solver_name, score in scoring_results.items()}

    # Record the solvers that had the top 5 scores    
    #top_5 = sorted(scoring_results.items(), key = lambda x: x[1])[:5]
    train_scoring_results[label] = sorted(scoring_results.items(), key = lambda x: x[1])

  0%|          | 0/400 [00:00<?, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x725bb9f22ab0>>
Traceback (most recent call last):
  File "/home/amzi/installs/miniforge3/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

  File "/home/amzi/installs/miniforge3/lib/python3.12/site-packages/timeout_decorator/timeout_decorator.py", line 69, in handler
    _raise_exception(timeout_exception, exception_message)
  File "/home/amzi/installs/miniforge3/lib/python3.12/site-packages/timeout_decorator/timeout_decorator.py", line 45, in _raise_exception
    raise exception()
timeout_decorator.timeout_decorator.TimeoutError: 'Timed Out'
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x725bb9f22ab0>>
Traceback (most recent call last):
  File "/home/amzi/installs/miniforge3/lib/python3.12/site-packag

In [9]:
sorted_scoring_results = sorted(train_scoring_results.items(), key = lambda x: [x[1][i][1] for i in range(5)])

In [10]:
sorted_scoring_results[:20]

[('c663677b',
  [('solve_0dfd9992', 0.0),
   ('solve_29ec7d0e', 0.0),
   ('solve_484b58aa', 0.0),
   ('verify_0dfd9992', 0.0),
   ('verify_29ec7d0e', 0.0),
   ('verify_484b58aa', 0.0),
   ('solve_73251a56', 0.010326488853131102),
   ('verify_73251a56', 0.012721063531307029),
   ('solve_9edfc990', 0.0198240890741843),
   ('verify_9edfc990', 0.0198240890741843),
   ('verify_3631a71a', 0.029708883601635455),
   ('verify_2204b7a8', 0.04034686488878521),
   ('verify_e48d4e1a', 0.05310572472223981),
   ('solve_a5313dff', 0.054594301839840646),
   ('verify_6cf79266', 0.060961636941665176),
   ('solve_6cf79266', 0.06259865185290626),
   ('solve_f8c80d96', 0.06390194298406039),
   ('solve_83302e8f', 0.06470047045577622),
   ('solve_3345333e', 0.06482218537438762),
   ('solve_a8d7556c', 0.06606295729294472),
   ('verify_a8d7556c', 0.06606295729294472),
   ('verify_ddf7fa4f', 0.07195995997334709),
   ('verify_1f642eb9', 0.0734776434608924),
   ('verify_4347f46a', 0.0743405720497154),
   ('solve_8

In [13]:
with open('baseline_solvers_vs_eval.json', 'w') as f:
    json.dump(sorted_scoring_results, f, indent = 4)